In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1.Load the data from the CSV file

In [ ]:
df_pgen1=pd.read_csv("../input/solar-power-generation-data/Plant_1_Generation_Data.csv")
df_psen1=pd.read_csv("../input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv")
df_pgen2=pd.read_csv("../input/solar-power-generation-data/Plant_2_Generation_Data.csv")
df_psen2=pd.read_csv("../input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv")


2Explore each dataset - columns, counts, basic stats

In [ ]:
df_pgen1

In [ ]:
df_pgen1.info()

In [ ]:
df_psen1

In [ ]:
df_psen1.info()

In [ ]:
df_pgen2[['PLANT_ID',"DATE_TIME",'AC_POWER']].value_counts().sort_index()

In [ ]:
df_pgen1.describe()

In [ ]:
len(df_psen1)

In [ ]:
(df_psen1.columns)

In [ ]:
len(df_psen1.columns)

In [ ]:
df_psen1.describe()

In [ ]:
df_psen1["IRRADIATION"].max()

In [ ]:
df_pgen1["AC_POWER"][670:5678].mean()

In [ ]:
df_pgen1.shape

In [ ]:
df_pgen1.shape[0]  #takes rows

In [ ]:
df_pgen1.shape[1]  #takes columns

In [ ]:
df_psen1

3.Understand the domain context and explore underlying patterns in the data

Plant_1_Generation_Data and Plant_2_Generation_Data files contain the information which are from the inverters. The files include data such as DC_POWER which is the amount of power absorbed by solar panels, AC_POWER which is converted from DC_POWER by inverters. Every inverter is associated with a unique key named SOURCE_KEY, which is used to mask the inverter ID. These files also contain information such as the daily yield of every inverter named DAILY_YIELD and the yield of an inverter in its entire duration named TOTAL_YIELD. The data is taken in intervals of every 15 minutes for 34 days.

Plant_1_Weather_Sensor_Data and Plant_2_Weather_Sensor_Data files contain the information which are from the various sensors in the solar plant. IRRADIATION contains the total exposure of a panel to sunlight.AMBIENT_TEMPERATURE contains the temperature of the surrounding and MODULE_TEMPERATURE contains the temperature of solar panel attached to the sensor panel. The SOURCE_KEY is a masked ID for the sensor panel. As there is only one sensor panel in the plant, there is same ID in the whole file. Here too, the data is taken in every 15 minutes interval for 34 days. 

All the data from sensors and inverters is passed to data logger(Gateway). The information can be stored in Clouds(Internet) for safe storage.



4.Explore the data and try to answer questions like - 

In [ ]:
#What is the mean value of daily yield?

print("The mean value of daily yield for plant 1 is " + str(df_pgen1["DAILY_YIELD"].mean()))
print("The mean value of daily yield for plant 2 is " + str(df_pgen2["DAILY_YIELD"].mean()))

In [ ]:
#What is the total irradiation per day?

# for this we need to separate the irradiation according to dates and then group by day to get the daily sum.

#For plant1

#DATE_TIME column is object type. We need to change to datetime type.
df_psen1["DATE_TIME"]=pd.to_datetime(df_psen1["DATE_TIME"], format='%Y-%m-%d %H:%M:%S')  

In [ ]:
df_psen1["DATE_TIME"].dtype # checking data type

In [ ]:
#creating a new column for date for separation according to dates
df_psen1["DATE"]=df_psen1["DATE_TIME"].dt.date
df_psen1                               

In [ ]:
group1=df_psen1.groupby(["DATE"]).sum()      # sum of all attributes with respect to date

In [ ]:
group1[["IRRADIATION"] ]# we need only IRRADIATION column from group1

In [ ]:
#For plant2

df_psen2["DATE_TIME"]=pd.to_datetime(df_psen2["DATE_TIME"], format='%Y-%m-%d %H:%M:%S')  

In [ ]:
df_psen2["DATE_TIME"].dtype

In [ ]:
df_psen2["DATE"]=df_psen2["DATE_TIME"].dt.date
df_psen2    

In [ ]:
group2=df_psen2.groupby(["DATE"]).sum()
group2[["IRRADIATION"] ]

In [ ]:
#What is the max ambient and module temperature?
#For plant1
print("The maximum value of ambient temperature for plant 1 is " + str(df_psen1["AMBIENT_TEMPERATURE"].max()))
print("The minimum value of ambient temperature for plant 1 is " + str(df_psen1["AMBIENT_TEMPERATURE"].min()))
print("The maximum value of module temperature for plant 1 is " + str(df_psen1["MODULE_TEMPERATURE"].max()))
print("The minimum value of module temperature for plant 1 is " + str(df_psen1["MODULE_TEMPERATURE"].min()))

#For plant2
print("The maximum value of ambient temperature for plant 2 is " + str(df_psen2["AMBIENT_TEMPERATURE"].max()))
print("The minimum value of ambient temperature for plant 2 is " + str(df_psen2["AMBIENT_TEMPERATURE"].min()))
print("The maximum value of module temperature for plant 2 is " + str(df_psen2["MODULE_TEMPERATURE"].max()))
print("The minimum value of module temperature for plant 2 is " + str(df_psen2["MODULE_TEMPERATURE"].min()))

In [ ]:
# How many inverters are there for each plant? 

print("There are "+str(len(df_pgen1["SOURCE_KEY"].unique()))+ " unique inverters for plant 1.")
print("There are "+str(len(df_pgen2["SOURCE_KEY"].unique()))+ " unique inverters for plant 2.")

In [ ]:
df_pgen1

In [ ]:
# What is the maximum/minimum amount of DC/AC Power generated in a time interval/day?

# we need to group by DC/AC Power and then find the minimum/maximum

#For plant1    found the maximun AC power for a day

#DATE_TIME column is object type. We need to change to datetime type.
df_pgen1["DATE_TIME"]=pd.to_datetime(df_pgen1["DATE_TIME"], format='%d-%m-%Y %H:%M')  
# storing date to different column
df_pgen1["DATE"]=df_pgen1['DATE_TIME'].dt.date
# displaying to check 
df_pgen1 

In [ ]:
# grouping according to date to find the value for a particular day
group3=df_pgen1.groupby(["DATE"]).max()

In [ ]:
#Displaying the AC_POWER column
group3[[ "AC_POWER"]]

In [ ]:
#For plant2   found the minimun DC power for a day
df_pgen2["DATE_TIME"]=pd.to_datetime(df_pgen2["DATE_TIME"], format='%Y-%m-%d %H:%M:%S')  
df_pgen2["DATE"]=df_pgen2['DATE_TIME'].dt.date
df_pgen2

In [ ]:
group4=df_pgen2.groupby(["DATE"]).min()
group4[["DC_POWER"]]

In [ ]:
# Which inverter (source_key) has produced maximum DC/AC power? 
# For plant1
print(df_pgen1[['SOURCE_KEY','AC_POWER']][df_pgen1.AC_POWER == df_pgen1.AC_POWER.max()])

In [ ]:
# For plant2
print(df_pgen2[['SOURCE_KEY','AC_POWER']][df_pgen2.AC_POWER == df_pgen2.AC_POWER.max()])

In [ ]:
#Rank the inverters based on the DC/AC power they produce

# For plant1
group5=df_pgen1.groupby(["SOURCE_KEY"]).sum()
group5


In [ ]:
group5.iloc[:,[2,3]].rank()

In [ ]:
# For plant2
group6=df_pgen2.groupby(["SOURCE_KEY"]).sum()
group6

In [ ]:
group6.iloc[:,[2,3]].rank()

In [ ]:
# Is there any missing data?

df_pgen1.isnull()

In [ ]:
df_pgen2.isnull()

In [ ]:
df_psen1.isnull()

In [ ]:
df_psen2.isnull()

In [ ]:
# There seems that no data is missing.